In [1]:
import pdbufr
import sys
import traceback
 
from eccodes import *
from ecmwf.opendata import Client
from math import isnan

from ipywidgets import interact
import os
import birdy
import geopandas as gpd
import pandas as pd
from datetime import datetime, timedelta
import ipyleaflet
import ipywidgets as widgets
from IPython.display import display

import panel as pn
pn.extension()

import warnings
warnings.filterwarnings("ignore")

In [2]:
## PICK STARTING DATE OF THE FORECAST ##

start_date_forecast = widgets.DatePicker(
    description = 'Forecast date:',
    value = datetime(datetime.now().year, datetime.now().month, datetime.now().day, hour=0),
)

start_date_forecast.style.description_width = '90px'

# Print widget
start_date_forecast

DatePicker(value=datetime.datetime(2023, 8, 18, 0, 0), description='Forecast date:', step=1, style=Description…

In [3]:
## DATA DOWNLOAD ##
# If too early in the morning the forecast of the current day cannot be donwloaded, need to fix this -> If raiseError: download forecast of the previous day
client = Client(source="azure")

client.retrieve(
    date=int(start_date_forecast.value.strftime("%Y%m%d")),
    time=0,
    stream="enfo",
    type="tf",
    step=240,
    target="track_data/tc_test_track_data.bufr",
);

20230818000000-240h-enfo-tf.bufr:   0%|          | 0.00/752k [00:00<?, ?B/s]

In [4]:
## SELECT CYCLONE TO PLOT ##

# Function to import storms file and load it in a dataframe
def create_storms_df():
    # Load cyclone dataframe with Mean sea level pressure value
    df_storms = pdbufr.read_bufr("track_data/tc_test.bufr",
        columns=("stormIdentifier", "ensembleMemberNumber", "latitude", "longitude",
                 "pressureReducedToMeanSeaLevel"))
    # Load cyclone dataframe with Wind speed at 10m value
    df1 = pdbufr.read_bufr("track_data/tc_test.bufr",
        columns=("stormIdentifier", "ensembleMemberNumber", "latitude", "longitude",
                 "windSpeedAt10M"))
    # Add the Wind speed at 10m column to the storms dataframe 
    df_storms["windSpeedAt10M"] = df1.windSpeedAt10M
    # Storms with number higher than 10 are not real storms (according to what Fernando said)
    drop_condition = df_storms.stormIdentifier < '11'
    df_storms = df_storms[drop_condition]
    return df_storms

df_storms = create_storms_df()
storms = df_storms.stormIdentifier.unique()

# Create widget for cyclone selection
cyclone = widgets.Dropdown(
    options = storms.tolist(),
    description = 'Active Storms:',
    disabled=False,
)
cyclone.style.description_width = '90px'

# Update storms list when downloading new tracks data
def update_storms_list(_):
    client.retrieve(
            date=int(start_date_forecast.value.strftime("%Y%m%d")),
            time=0,
            stream="enfo",
            type="tf",
            step=240,
            target="track_data/tc_test.bufr",
        );
    df_storms = create_storms_df()
    storms = df_storms.stormIdentifier.unique()
    cyclone.options = storms.tolist()

start_date_forecast.observe(update_storms_list, names='value')

# Print widget
display(cyclone)

Dropdown(description='Active Storms:', options=('07E', '08E', '09E'), style=DescriptionStyle(description_width…

In [5]:
## UPDATE BUTTON TO DOWNLOAD NEW DATA AND CREATE STORM DATAFRAME DEPENDING ON THE CYCLONE SELECTED ##

update_button = widgets.Button(
    description = 'Update Forecast',
    style=dict(
        button_color='blue',
        font_weight='bold',
        text_color='lightgreen',
        text_decoration='underline',
))

update_output = widgets.Output()

# Download new forecast data when clicking the Update Forecast Button
def on_button_clicked(b):
    with update_output:
        client.retrieve(
            date=int(start_date_forecast.value.strftime("%Y%m%d")),
            time=0,
            stream="enfo",
            type="tf",
            step=240,
            target="track_data/tc_test.bufr",
        );
        df_storms = create_storms_df()
        storms = df_storms.stormIdentifier.unique()
        cyclone.options = storms.tolist()
        print("Forecast updated!", end='\r')

update_button.on_click(on_button_clicked)

# Print widget
display(update_button, update_output)

Button(description='Update Forecast', style=ButtonStyle(button_color='blue', font_weight='bold', text_color='l…

Output()

In [7]:
## SELECT ENSEMBLE MEMBER TO PLOT ##
df_storms = create_storms_df()
df_cyclone = df_storms[df_storms.stormIdentifier == cyclone.value]
members = df_cyclone.ensembleMemberNumber.unique()

# Create widget for ensemble member selection
member = widgets.Dropdown(
    options = members.tolist(),
    value = 1,
    description = 'Ensemble member:',
    disabled=False,
)
member.style.description_width = '120px'

# Link ensemble selection widget to cyclone selection widget
def update_ensemble_members(_):
    df_storms = create_storms_df()
    df_cyclone = df_storms[df_storms.stormIdentifier == cyclone.value]
    members = df_cyclone.ensembleMemberNumber.unique()
    member.options = members.tolist()
    member.value=1

update_button.on_click(update_ensemble_members)
cyclone.observe(update_ensemble_members, names='value')

# Print widget
display(member)

Dropdown(description='Ensemble member:', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1…

In [8]:
## TRACK DATAFRAME CREATION ##

# Function to create track dataframe according to the forecast start date, cyclone and ensemble member selected
def create_track_df(df_cycl, start_date, ens_member):
    df_track = df_cycl[df_cycl.ensembleMemberNumber == ens_member]
    
    # Add column of dates to the dataframe
    df_track.reset_index(inplace=True)
    df_track.drop(columns="index", inplace=True)
    start = datetime(start_date.year, start_date.month, start_date.day, hour=0)
    df_track["date"] = start + (df_track.index.to_numpy()+1) * timedelta(hours=6)
    
    # Drop NaN in latitude and longitude columns
    df_track.dropna(subset = ['latitude', 'longitude'], inplace=True)
    
    # Locations and dates list for temporal horizon widget
    lat = df_track.latitude.values
    lon = df_track.longitude.values
    locations = [[lat[i], lon[i]] for i in range(len(lat))]
    track_dates = df_track["date"]
    
    return locations, track_dates

locations, track_dates = create_track_df(df_cyclone, start_date_forecast.value, member.value)

In [9]:
## TEMPORAL HORIZON RANGE SLIDEBAR WITH DATES ##
options = [(timestep.strftime('%d/%m %H'), timestep) for timestep in track_dates]
index = (0, len(options)-1)

temporal_horizon_slider = widgets.SelectionRangeSlider(
    options=options,
    index=index,
    # description='Temporal Horizon:',
    description='Define horizon:',
    orientation='horizontal',
    layout={'width': '500px'},
    disabled=False,
)

# Define the size of the descprition box of the slider and other styles
temporal_horizon_slider.style.description_width = '100px'
temporal_horizon_slider.style.handle_color = 'lightgreen'

def print_date_range(date_range):
    start, end = date_range
    s = start.strftime('%d %b %Y %H:%M')
    e = end.strftime('%d %b %Y %H:%M')
    print(f'Temporal horizon: {s} - {e}')
    
# Link horizon slider to ensemble member selection
def update_horizon_ens(_):
    df_storms = create_storms_df()
    df_cyclone = df_storms[df_storms.stormIdentifier == cyclone.value]
    locations, track_dates = create_track_df(df_cyclone, start_date_forecast.value, member.value)
    options = [(timestep.strftime('%d/%m %H'), timestep) for timestep in track_dates]
    temporal_horizon_slider.options = options
    temporal_horizon_slider.index = (0, len(options)-1)

cyclone.observe(update_horizon_ens, names='value')
member.observe(update_horizon_ens, names='value')
start_date_forecast.observe(update_horizon_ens, names='value')

# Print widget with selected temporal horizon displayed
temporal_horizon_slider
widgets.interact(
        print_date_range,
        date_range=temporal_horizon_slider
    );

interactive(children=(SelectionRangeSlider(description='Define horizon:', index=(0, 9), layout=Layout(width='5…

In [10]:
## TEMPORAL SLIDEBAR TO SELECT SNAPSHOT##
# Function to convert datetime to integer
def datetime_to_int(dt):
    return int((dt - start_date_forecast.value - timedelta(hours=6)).total_seconds() // timedelta(hours=6).total_seconds())

# Function to convert interger to datetime
def int_to_datetime(ts):
    return start_date_forecast.value + timedelta(hours=6) + ts * timedelta(hours=6)

timestep_slider=widgets.IntSlider(
    min=0, 
    max=datetime_to_int(track_dates.iloc[-1].to_pydatetime()), 
    step=1, 
    value=0, 
    description='Timestep:')

# Increase the size of the descprition box of the slider and other styles
timestep_slider.style.description_width = '65px'
timestep_slider.style.handle_color = 'lightgreen'
    
def print_timestep(dt):
    date = int_to_datetime(dt)
    print('Selected date: ', date.strftime('%d %b %Y %H:%M'))
    
# Link the timestep selection slider to the horizon slider
def update_snapshot_slider(_):
    start, end = temporal_horizon_slider.value
    timestep_slider.value = datetime_to_int(start.to_pydatetime())
    timestep_slider.min = datetime_to_int(start.to_pydatetime())
    timestep_slider.max = datetime_to_int(end.to_pydatetime())

temporal_horizon_slider.observe(update_snapshot_slider, names='value')

# Print widget with selected date displayed
timestep_slider
widgets.interact(
    print_timestep,
    dt=timestep_slider
);

interactive(children=(IntSlider(value=0, description='Timestep:', max=10, style=SliderStyle(description_width=…

In [11]:
## INTERACTIVE MAP FOR TROPICAL CYCLONE TRACK ##
# Create the basemap for plotting
initial_lat_lon = (locations[0][0], locations[0][1])
tc_track_map = ipyleaflet.Map(
    center=initial_lat_lon,
    basemap=ipyleaflet.basemaps.OpenStreetMap.France,
    zoom = 3.5,
)

# Add tropical cyclone track to the map
track = ipyleaflet.Polyline(
    locations=locations,
    color="blue" ,
    fill=False,
    name='Track',
)

# Link the cyclone track to the ensemble number and the temporal horizon 
def update_track(_):
    df_storms = create_storms_df()
    df_cyclone = df_storms[df_storms.stormIdentifier == cyclone.value]
    locations, track_dates = create_track_df(df_cyclone, start_date_forecast.value, member.value)
    start, end = temporal_horizon_slider.value
    start_index = datetime_to_int(start.to_pydatetime())
    end_index = datetime_to_int(end.to_pydatetime())
    track.locations = locations[start_index:end_index+1]

update_button.on_click(update_track)
cyclone.observe(update_track, names='value')
member.observe(update_track, names='value')
temporal_horizon_slider.observe(update_track, names='value')
start_date_forecast.observe(update_track, names='value')

tc_track_map.add_layer(track)

# Add a marker to the map, for the snapshot selected
marker = ipyleaflet.CircleMarker(location=locations[timestep_slider.value],
                                 draggable=False,
                                 name = 'Timestep',
                                 color = "lightgreen",
                                 fill_color = "lightgreen",
                                 radius = 2,
                                )

# Link the marker to the snapshot selections slider
def update_marker_location(_):
    df_storms = create_storms_df()
    df_cyclone = df_storms[df_storms.stormIdentifier == cyclone.value]
    locations, track_dates = create_track_df(df_cyclone, start_date_forecast.value, member.value)
    marker.location = locations[timestep_slider.value]

update_button.on_click(update_marker_location)
cyclone.observe(update_marker_location, names='value') 
member.observe(update_marker_location, names='value')
timestep_slider.observe(update_marker_location, names='value')
start_date_forecast.observe(update_marker_location, names='value')

tc_track_map.add_layer(marker)

# Add layers widget to the map
tc_track_map.add_control(ipyleaflet.LayersControl());

# Print map
tc_track_map

Map(center=[16.2, -134.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…